In [ ]:
!pip install gensim

In [1]:
import gensim
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

In [2]:
text = pd.read_csv("IMDB Dataset.csv")

In [3]:
text.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
text["review_clean"] = text["review"].apply(lambda x: gensim.utils.simple_preprocess(x))
text.head()

,review,sentiment,review_clean
0,One of the other reviewers has mentioned that ...,positive,"[one, of, the, other, reviewers, has, mentione..."
1,A wonderful little production. <br /><br />The...,positive,"[wonderful, little, production, br, br, the, f..."
2,I thought this was a wonderful way to spend ti...,positive,"[thought, this, was, wonderful, way, to, spend..."
3,Basically there's a family where a little boy ...,negative,"[basically, there, family, where, little, boy,..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[petter, mattei, love, in, the, time, of, mone..."


In [5]:
clean_text = text[["review_clean"]]
labels = text["sentiment"]

In [6]:
len(clean_text)

50000

In [7]:
corpus = []
for review in clean_text["review_clean"]:
  corpus.append(" ".join(review))

In [8]:
len(corpus)

50000

In [9]:
clean_text["corpus"] = corpus

C:\Users\bmaj3\AppData\Local\Temp\ipykernel_7360\1042297266.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_text["corpus"] = corpus


In [10]:
len(clean_text)

50000

In [11]:
clean_text

,review_clean,corpus
0,"[one, of, the, other, reviewers, has, mentione...",one of the other reviewers has mentioned that ...
1,"[wonderful, little, production, br, br, the, f...",wonderful little production br br the filming ...
2,"[thought, this, was, wonderful, way, to, spend...",thought this was wonderful way to spend time o...
3,"[basically, there, family, where, little, boy,...",basically there family where little boy jake t...
4,"[petter, mattei, love, in, the, time, of, mone...",petter mattei love in the time of money is vis...
...,...,...
49995,"[thought, this, movie, did, down, right, good,...",thought this movie did down right good job it ...
49996,"[bad, plot, bad, dialogue, bad, acting, idioti...",bad plot bad dialogue bad acting idiotic direc...
49997,"[am, catholic, taught, in, parochial, elementa...",am catholic taught in parochial elementary sch...
49998,"[going, to, have, to, disagree, with, the, pre...",going to have to disagree with the previous co...


In [12]:
X_train, X_test, y_train, y_test = train_test_split(clean_text, 
                                                    labels, 
                                                    test_size=0.30, 
                                                    random_state=42)

In [13]:
tfidf_vect = TfidfVectorizer()
X_train_tfidf = tfidf_vect.fit_transform(X_train["corpus"])
X_test_tfidf = tfidf_vect.transform(X_test["corpus"])

In [ ]:
svd = TruncatedSVD(n_components = 10000, n_iter=7, random_state=42)
X_train_reduced = svd.fit_transform(X_train_tfidf)
X_test_reduced = svd.transform(X_test_tfidf)

In [ ]:
sum(svd.explained_variance_)

In [ ]:
# 5000 SVD components takes 67 minutes and 34 seconds 

In [ ]:
X_train_tfidf_df = pd.DataFrame(X_train_reduced, index = X_train.index, columns = [i for i in range(101, 5101)])
X_test_tfidf_df = pd.DataFrame(X_test_reduced, index = X_test.index, columns = [i for i in range(101, 5101)])

In [ ]:
w2v_model = gensim.models.Word2Vec(X_train["review_clean"],
                                   vector_size=100,
                                   window=5,
                                   min_count=2)

In [ ]:
words = set(w2v_model.wv.index_to_key)
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words]) for ls in X_train["review_clean"]], dtype = "object")
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words]) for ls in X_test["review_clean"]], dtype = "object")

In [ ]:
# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

In [ ]:
X_train_df = pd.DataFrame(X_train_vect_avg, index = X_train.index)
X_test_df = pd.DataFrame(X_test_vect_avg, index = X_test.index)

In [ ]:
X_train = pd.concat([X_train_df, X_train_tfidf_df], axis = 1)
X_test = pd.concat([X_test_df, X_test_tfidf_df], axis = 1)

In [ ]:
X_train

In [ ]:
#X_train.to_csv("X_train.csv",index = False)

In [ ]:
#X_test.to_csv("X_test.csv",index = False)
#y_train.to_csv("y_train.csv", index = False)
#y_test.to_csv("y_test.csv",index = False)

In [ ]:
clf = SVC()
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print(accuracy_score(y_test, y_pred))

In [2]:
from sklearn.neural_network import MLPClassifier

In [3]:
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")
y_test = pd.read_csv("y_test.csv")

In [24]:
mlp_clf = MLPClassifier(random_state=1, 
                        max_iter=300, 
                        verbose = True, 
                        learning_rate = "adaptive",
                        hidden_layer_sizes= (500,))


In [25]:
mlp_clf.fit(X_train, y_train)

c:\Users\bmaj3\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.40474168
Iteration 2, loss = 0.24661517
Iteration 3, loss = 0.20853547
Iteration 4, loss = 0.19124095
Iteration 5, loss = 0.17913470
Iteration 6, loss = 0.17258685
Iteration 7, loss = 0.15931708
Iteration 8, loss = 0.14822907
Iteration 9, loss = 0.13690262
Iteration 10, loss = 0.12244667
Iteration 11, loss = 0.11046435
Iteration 12, loss = 0.09495594
Iteration 13, loss = 0.07900742
Iteration 14, loss = 0.06443615
Iteration 15, loss = 0.05225513
Iteration 16, loss = 0.04103373
Iteration 17, loss = 0.03338778
Iteration 18, loss = 0.02623813
Iteration 19, loss = 0.02062841
Iteration 20, loss = 0.01708508
Iteration 21, loss = 0.01437299
Iteration 22, loss = 0.01240667
Iteration 23, loss = 0.01106241
Iteration 24, loss = 0.01000019
Iteration 25, loss = 0.00915337
Iteration 26, loss = 0.00844007
Iteration 27, loss = 0.00798715
Iteration 28, loss = 0.00755547
Iteration 29, loss = 0.00716364
Iteration 30, loss = 0.00688550
Iteration 31, loss = 0.00662502
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(500,), learning_rate='adaptive', max_iter=300,
              random_state=1, verbose=True)

In [28]:
# 500 neurons 1 layer took 22 minutes 39 seconds

In [26]:
y_predict = mlp_clf.predict(X_test)

In [27]:
print(accuracy_score(y_test, y_predict))

0.8930666666666667


In [17]:
mlp_clf.n_layers_

4